# 基础理论部分

0. Can you come up out 3 sceneraies which use AI methods?
（1）语音助手（2）自动驾驶（3）机器人客服

1. How do we use Github; Why do we use Jupyter and Pycharm;
Github是一个开源社区，里面有很多优秀的开源项目，其中包含很多AI相关的项目，所以可以在Github查找、学习和使用这些项目。
Jupyter and Pycharm是python IDE工具，使用这些工具有助于快速的开发。Jupyter基于IPython提供便捷的交互式开发环境，各种快捷方式，魔术命令。Pycharm主要用于开发大型的python项目。

2. What's the Probability Model?
以概率论为基础，使用数学方法描述语言的规律

3. Can you came up with some sceneraies at which we could use Probability Model?
1、垃圾邮件过滤

4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match?
用概率来衡量一个句子是否合理能够把复杂的问题变得简单化，编写的代码具有通用性，不必因为规则的改变而修改代码。
基于解析和模式匹配的编程不具有通用性和扩展性，一旦规则发生改变就需要修改代码

5. What's the Language Model;
为自然语言上下文相关的特性建立数学模型

6. Can you came up with some sceneraies at which we could use Language Model?
1、语音识别
2、机器翻译

7. What's the 1-gram language model;
1-gram language model是一个上下文无关的模型，假设当前出现的概率与前面的词无关

8. What's the disadvantages and advantages of 1-gram language model;
1-gram language model的缺点是效果不明显，准确率低
1-gram language model的优点是易于理解和计算

9. What't the 2-gram model;
2-gram model是指句子中的每个词之和前面的一个词有关，即在已知第一个词前提下，第二个词出现的概率

In [1]:
import random
def generate_sentence(grammar_rules: str, target, stmt_split='=', expr_split='|'):
    """
    产生句子
    """
    grammar_rules_dict = parse_grammar_rules(grammar_rules, stmt_split, expr_split)

    return random_genrate_sentence(grammar_rules_dict, target)

def parse_grammar_rules(grammar_rules: str, stmt_split='=', expr_split='|'):
    """
    解析语法规则
    """
    grammar_rules_dict = {}
    for line in grammar_rules.split('\n'):
        if line:
            stmt, expr = line.split(stmt_split)
            grammar_rules_dict[stmt.strip()] = expr.split(expr_split)
    return grammar_rules_dict;

def random_genrate_sentence(grammar_rules: dict, target):
    """
    随机产生句子
    """
    if target in grammar_rules:
        candidate = random.choice(grammar_rules[target])
        return ''.join(random_genrate_sentence(grammar_rules, e.strip()) for e in candidate.split())
    else:
        return target

In [2]:
import re
import zhon.hanzi
def filter_invalid_char(string: str):
    chinese_punctuation = zhon.hanzi.punctuation
    english_punctuation = ',.?;:\'"`~!'
    special_char = r'<>/\\|\[\]{}@#\$%\^&\*\(\)-_\+=\n'
    return re.sub('(?P<punctuation>[{}]|[{}])|(?P<special_char>[{}])'.format(chinese_punctuation, english_punctuation, special_char), '', string)

In [3]:
filter_invalid_char('很燃！战争的场,面..很?$%()宏伟，人与人的打and machine,any斗很+++---精彩！看得人热血沸腾，为同胞而战！\n')

'很燃战争的场面很宏伟人与人的打and machineany斗很精彩看得人热血沸腾为同胞而战'

In [4]:
filter_invalid_char('')

''

In [5]:
import csv
def clean_data_set(movie_comments_file_path, train_file_path, output_file_path):
    with open(movie_comments_file_path, encoding='utf-8') as movie_comments_file, open(train_file_path, encoding='utf-8') as train_file, open(output_file_path, 'w', encoding='utf-8') as output_file:
        movie_comments_reader = csv.DictReader(movie_comments_file)
        for row in movie_comments_reader:
            output_file.write(filter_invalid_char(row['comment']))
            
        for line in train_file:
            output_file.write(filter_invalid_char(line))

In [6]:
import jieba
from collections import Counter

def generate_2_gram_words_count(sample_file_path: str, unit_length=1000000):
    words_count = {}
    two_gram_words_count = {}
    with open(sample_file_path, encoding='utf-8') as sample_file:
        sample = sample_file.read()
        begin_index = 0
        while begin_index < len(sample):
            words = cut(sample[begin_index:(begin_index + unit_length)])
            _generate_2_gram_words_count(words, words_count, two_gram_words_count)
            begin_index = begin_index + unit_length
    return words_count, two_gram_words_count

def cut(sentence):
    return list(jieba.cut(sentence))

def _generate_2_gram_words_count(words: list, words_count: dict, two_gram_words_count: dict):
    _words_count = Counter(words)
    for w, f in _words_count.most_common():
        if w in words_count:
            words_count[w] = words_count[w] + f
        else:
            words_count[w] = f
    
    tow_gram_words = [words[i] + words[i + 1] for i in range(0, len(words) - 1)]
    _tow_gram_words_count = Counter(tow_gram_words)
    for tgw, f in _tow_gram_words_count.most_common():
        if tgw in two_gram_words_count:
            two_gram_words_count[tgw] = two_gram_words_count[tgw] + f
        else:
            two_gram_words_count[tgw] = f
            
def calculate_probability_of_sentence_by_two_gram(sentence: str, words_count: dict, two_gram_words_count: dict):
    """
    Add-one (Laplace) Smoothing
    """
    words = cut(sentence)
    probability = 1
    v = len(words_count)
    probability *= (words_count[words[0]] if words[0] in words_count else 0  + 1) / (sum(words_count.values()) + v)
    for i in range(0, len(words) - 1):
        w1w2 = words[i] + words[i+1]
        probability *= (two_gram_words_count[w1w2] if w1w2 in two_gram_words_count else 0 + 1) / (words_count[words[i]] if words[i] in words_count else 0 + v)
    return probability

In [7]:
movie_comments_file_path = 'D:\\Users\\ZengYubin\\study\\nlp-coures\\lesson01\\assignment\\data\\movie_comments.csv'
train_file_path = 'D:\\Users\\ZengYubin\\study\\nlp-coures\\lesson01\\assignment\\data\\train.txt'
output_file_path = 'D:\\Users\\ZengYubin\\study\\nlp-coures\\lesson01\\assignment\\data\\output.txt'
clean_data_set(movie_comments_file_path, train_file_path, output_file_path)

In [8]:
words_count, two_gram_words_count = generate_2_gram_words_count(output_file_path)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ZENGYU~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.121 seconds.
Prefix dict has been built succesfully.


In [9]:
print(len(words_count))
print(list(words_count.keys())[:100])
print(len(two_gram_words_count))
print(list(two_gram_words_count.keys())[:100])

163552
['的', '了', ' ', '是', '我', '都', '电影', '看', '也', '和', '在', '很', '不', '有', '就', '人', '好', '啊', '你', '还', '这', '一个', '还是', '但', '没有', '让', '就是', '剧情', '故事', '说', '喜欢', '太', '又', '他', '给', '没', '吧', '到', '上', '得', '一部', '能', '对', '被', '最后', '最', '这个', '多', '真的', '什么', '不错', '要', '不是', '可以', '觉得', '好看', '自己', '拍', '这部', '想', '感觉', '去', '把', '与', '演技', '导演', '里', '中', '会', '这么', '着', '但是', '片子', '那', '更', '大', '那么', '挺', '有点', '再', '这种', '小', '比', '这样', '真是', '而', '个', '时候', '不过', '演员', '吗', '我们', '虽然', '爱', '看到', '来', '知道', '片', '却', '戏']
1766417
['  ', '的电影', '都是', '让人', '看的', '也是', '的人', '看了', '的是', '的时候', '的故事', '看完', '我的', '了 ', '让我', '这部电影', '的感觉', '了我', '好的', '这样的', '不知道', '很好', '电影的', '人的', '这是', '我觉得', '自己的', '是我', '的片子', '的一部', '拍的', '的剧情', '★★', '喜欢的', '的 ', '中的', '他的', '是个', '的演技', '好看的', '了吧', '都有', '最好的', '还不错', '我就', '我也', '的我', '都很', '去看', '很有', '我是', '真的是', '不错的', '的青春', '你的', '也很', '的好', ' 我', '不喜欢', '人都', '里的', '最后的', '的爱情', '也不', '一个人', '上的', '的都', '多了', '才是', '看得',

In [10]:
grammar_one = """
角色活动 = 角色 动作1 地点 动作2 角色 内容
角色 = 老师 | 学生 | 医生 | 病人
动作1 = 在 | 来 | 回 | 去
地点 = 学校 | 教室 | 医院 | 病房
动作2 = 教 | 听 | 看 | 为 | 请教 | 询问 | 帮
内容 = 学习 | 上课 | 看病 | 做作业 | 病情 | 解决问题 | 读书
"""
grammar_tow = """
智能客服 = 打招呼 介绍 询问
打招呼 = 您好 | 你好 | 早上好 | 下午好 | 晚上好 | 晚安 | 早安
介绍 = 我是 名称 | 名称 为你服务
名称 = 小美 | 小冰 | 小娜 | 小爱
询问 = 称呼 有什么需要 事情 | 称呼 有什么 事情 | 我能为 称呼 做什么 | 称呼 需要做什么 | 请赐教
称呼 = 你 | 您
事情 = 帮忙的 | 查询的 | 困难 | 困惑 
"""

In [11]:
generate_sentence(grammar_one, '角色活动')

'学生去病房询问老师做作业'

In [12]:
generate_sentence(grammar_tow, '智能客服')

'晚安我是小娜我能为您做什么'

In [13]:
def generate_n_sentences(n):
    grammars = [{'grammar_rules': grammar_one, 'target': '角色活动'}, {'grammar_rules': grammar_tow, 'target': '智能客服'}]
    sentences = []
    for _ in range(0, n):
        grammar = random.choice(grammars)
        sentences.append(generate_sentence(grammar['grammar_rules'], grammar['target']))
    return sentences

def calculate_probability_of_sentence(sentence: str):
    return calculate_probability_of_sentence_by_two_gram(sentence, words_count, two_gram_words_count)

def generate_best_sentence(n=10):
    probability_of_sentences = [(sentence, calculate_probability_of_sentence(sentence)) for sentence in generate_n_sentences(n)]
    print(probability_of_sentences)
    return sorted(probability_of_sentences, key=lambda p: p[1], reverse=True)[0]

In [14]:
generate_n_sentences(10)

['病人来教室教学生读书',
 '学生回病房听学生病情',
 '晚上好小美为你服务您有什么需要查询的',
 '晚安小爱为你服务我能为您做什么',
 '晚安我是小冰请赐教',
 '早安小冰为你服务我能为你做什么',
 '学生来学校教病人读书',
 '学生在病房为老师读书',
 '早安小冰为你服务你需要做什么',
 '老师在教室看学生学习']

In [15]:
sentence = generate_sentence(grammar_one, '角色活动')
print(sentence)
print(calculate_probability_of_sentence(sentence))

病人来学校询问学生看病
5.543775988564482e-17


In [16]:
generate_best_sentence()

[('老师来医院听病人病情', 1.5270346492099765e-17), ('早安小美为你服务请赐教', 6.489086577600779e-22), ('您好我是小娜您有什么需要困难', 1.9259533807049488e-25), ('早上好我是小冰您有什么困难', 7.028766862882715e-22), ('晚安我是小冰你需要做什么', 1.5405350203473182e-20), ('晚上好我是小美我能为你做什么', 1.4254361285541643e-27), ('晚上好小美为你服务你有什么查询的', 3.570810891794637e-27), ('晚安小冰为你服务请赐教', 6.489086577600779e-22), ('病人去医院听病人学习', 2.476591768007258e-17), ('老师去学校听医生学习', 7.344807402206046e-19)]


('病人去医院听病人学习', 2.476591768007258e-17)

# 这个模型有什么问题？ 你准备如何提升？
这个模型的效果不够好，比如“早上好我是小冰您有什么困难”比“学生去医院为病人病情”更接近人类的语言。
出现这个问题的原因可能是数据集不够大或者需要改进语言模型：
数据集不够大，可以增加数据集进行训练
改进语言模型，可以使用3-Gram模型、修改平滑算法等（在这个模型中使用的是Add-one (Laplace) Smoothing）